In [24]:
import sys
import numpy as np
import os
sys.path.append(os.path.expanduser("~/git/machine_learning/src"))
import importlib
import models
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
sys.path.insert(0,os.path.expanduser("~/git/machine_learning/src"))

In [25]:
data_dir = os.path.expanduser("~m2d/git/analysis_playground/ml/data/training_data")

pars = np.load(os.path.join(data_dir, "orhs3-noise_pars.npy"))[:3000000]
refl = np.load(os.path.join(data_dir, "orhs3-noise_data.npy"))[:3000000]
q_values = np.load(os.path.join(data_dir, "orhs3-noise_q_values.npy"))

print(pars.shape)
print(refl.shape)
print(q_values.shape)

idx = 0
nset=10000

(1999998,)
(1999998, 100)
(100,)


In [26]:
#for pars prediction
print(refl[0].shape)
r=refl[0][np.newaxis]
r=r * r.T
print(r)

(100,)
[[ 6.19344761e-05  1.58571115e-04  3.42658074e-04 ... -2.79985906e-02
  -2.76445514e-02 -2.72850379e-02]
 [ 1.58571115e-04  4.05990331e-04  8.77308991e-04 ... -7.16849161e-02
  -7.07784679e-02 -6.98580040e-02]
 [ 3.42658074e-04  8.77308991e-04  1.89578669e-03 ... -1.54904727e-01
  -1.52945973e-01 -1.50956932e-01]
 ...
 [-2.79985906e-02 -7.16849161e-02 -1.54904727e-01 ...  1.26572650e+01
   1.24972152e+01  1.23346907e+01]
 [-2.76445514e-02 -7.07784679e-02 -1.52945973e-01 ...  1.24972152e+01
   1.23391892e+01  1.21787198e+01]
 [-2.72850379e-02 -6.98580040e-02 -1.50956932e-01 ...  1.23346907e+01
   1.21787198e+01  1.20203373e+01]]


In [27]:


testset= np.asarray(refl[-nset:])
testsetout= np.asarray(pars[-nset:])
testpars = np.asarray (pars[-nset:])


In [ ]:
import importlib
importlib.reload(models)
n_data = refl.shape[0]
n_features = refl.shape[1]
n_outputs = 1

encoder=models.cnn_encoder(n_data, n_features, n_outputs)
encoder.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam())
history = encoder.fit(refl, pars,
                        epochs=20, batch_size=2056,
                        validation_data=(testset, testsetout))

2023-06-22 15:28:35.275438: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/20
973/973 [==============================] - 133s 136ms/step - loss: 0.4013 - val_loss: 0.3278
Epoch 2/20
973/973 [==============================] - 139s 143ms/step - loss: 0.3117 - val_loss: 0.2971
Epoch 3/20
973/973 [==============================] - 131s 134ms/step - loss: 0.2854 - val_loss: 0.2688
Epoch 4/20
973/973 [==============================] - 134s 138ms/step - loss: 0.2654 - val_loss: 0.2543
Epoch 5/20
973/973 [==============================] - 135s 138ms/step - loss: 0.2500 - val_loss: 0.2408
Epoch 6/20
973/973 [==============================] - 134s 138ms/step - loss: 0.2428 - val_loss: 0.2379
Epoch 7/20
973/973 [==============================] - 129s 133ms/step - loss: 0.2380 - val_loss: 0.2321
Epoch 8/20
973/973 [==============================] - 133s 137ms/step - loss: 0.2333 - val_loss: 0.2370
Epoch 9/20
973/973 [==============================] - 139s 143ms/step - loss: 0.2297 - val_loss: 0.2267
Epoch 10/20
973/973 [==============================] - 130s 134m

In [18]:
fig = plt.figure(dpi=100, figsize=[8,4])

plt.plot(np.arange(len(history['loss'])), history['loss'], label='loss')
plt.plot(np.arange(len(history['val_loss'])), history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.xscale('linear')
plt.show()

NameError: name 'history' is not defined

<Figure size 800x400 with 0 Axes>

In [6]:
importlib.reload(models)
data_dir = os.path.expanduser("~/git/machine_learning/data")
models.save_model(encoder, "cnn3-noise", data_dir)

In [28]:
importlib.reload(models)
data_dir = os.path.expanduser("~/git/machine_learning/data")

In [29]:
model=models.load_model("cnn3-noise", data_dir)

In [30]:
preds=model.predict(testset[-1000:])
_preds = [np.around(p) for p in preds]
models.accuracy(testpars[-1000:], _preds)

32/32 [==============================] - 6s 152ms/step
[256, 61, 0, 0]
Layer Accuracy : 0.807571
[25, 191, 120, 0]
Layer Accuracy : 0.568452
[1, 83, 263, 0]
Layer Accuracy : 0.757925
[0, 0, 0, 0]
Layer Accuracy : nan
Accuracy: 0.71


/SNS/users/vogtdan000/git/machine_learning/src/models.py:141: RuntimeWarning: invalid value encountered in long_scalars
  print("Layer Accuracy : %g" % (k/f))
